# Modul 3: Sentiment Analysis
Sentiment Analysis adalah proses melakukan identifikasi dan ekstrak informasi dari data teks untuk mengetahui sentimen dari produk yang bisa berupa positif, negatif, atau netral. Berikut ini adalah contohnya dari review film Spider-Man: Into the Spider-Verse:

* positif: "Keren banget animasinya, film animasi terbaik yang pernah saya tonton!"
* negatif: "Biasa aja gak ada seru-serunya, alur ceritanya jelek."
* netral: "Pertamax gan!"

## Step 1: Explore Data
Dataset yang digunakan adalah data hasil crawling dari Bukalapak, yang setiap datanya sudah terdapat column rate dari 1-5.

In [ ]:
# lakukan load dataset terlebih dahulu
import pandas as pd

# TODO: load dataset

In [ ]:
# coba kita perhatikan berapa banyak data yang kita dapatkan
# TODO: jalankan perintah .describe()

In [ ]:
# coba kita lihat salah satu hasil review
df_review["review"][3]

Salah satu tantangan dalam data kita kali ini adalah banyak bahasa yang tidak lengkap dan tidak baku.

In [ ]:
# coba kita lihat berapa banyak jumlah data untuk tiap rate
# TODO: gunakan .value_counts() untuk lihat jumlah data tiap rate

In [ ]:
# coba kita lihat user unik yang ada
# TODO: gunakan .value_counts() untuk lihat jumlah data tiap nama

In [ ]:
# coba kita lihat salah satu user, komentar apa yang dia kemukakan
# TODO: tampilkan komentar salah satu dari top user

## Step 2: Preprocessing
Pada tahap ini tujuannya adalah mengubah dataset supaya dapat diolah pada algoritma Machine Learning. 

Pertama, kita akan mengubah menjadi 2 sentiment dulu, yaitu 'positif' dan 'negatif'.
* Untuk sentiment positif, adalah data dengan rate 5, menjadi label 1
* Untuk sentiment negatif, adalah data dengan rate 1-4, menjadi label 0

In [ ]:
# kita lihat dulu distribusi label
import matplotlib.pyplot as plt

_, ax1 = plt.subplots(figsize=(12,6))

stars_histogram = df_review["rate"].value_counts().sort_index()

stars_histogram.plot(kind="bar", width=1.0)
plt.tight_layout()
plt.show()

In [ ]:
# mengubah rate
label = []
# TODO: buat perulangan for untuk ganti label

df_review["label"] = label
df_review = df_review.drop(columns=['rate'])
df_review.tail()

In [ ]:
# cek lagi data
df_review['label'].value_counts()

In [ ]:
# kita lihat lagi distribusi label
import matplotlib.pyplot as plt

_, ax1 = plt.subplots(figsize=(12,6))

stars_histogram = df_review["label"].value_counts().sort_index()

stars_histogram.plot(kind="bar", width=1.0)
plt.tight_layout()
plt.show()

### Cleansing Data
Hapus tanda baca, lower text, dll.

In [ ]:
import string, re

def cleansing(data):
    # lower text
    # TODO: buat lower teksnya
    
    # hapus punctuation
    # TODO: hapus punctiation
    
    # remove ASCII dan unicode
    data = data.encode('ascii', 'ignore').decode('utf-8')
    data = re.sub(r'[^\x00-\x7f]',r'', data)
    
    # remove newline
    # TODO: hapus newline
    
    return data

In [ ]:
# jalankan cleansing data
review = []
for index, row in df_review.iterrows():
    review.append(cleansing(row["review"]))
    
df_review["review"] = review
df_review.head()

### Menghapus Stopwords
Berikutnya, kita akan menghapus stopwords menggunakan library Sastrawi. Stopwords adalah kata-kata yang tidak memiliki makna.

In [ ]:
# install sastrawi terlebih dahulu dengan pip install Sastrawi

In [ ]:
# import library
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
 
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
 
# Contoh
kalimat = 'Dengan Menggunakan Python dan Library Sastrawi saya dapat melakukan proses Stopword Removal'
stop = stopword.remove(kalimat)
print(stop)

In [ ]:
# lakukan pada data kita

review = []
# TODO: lakukan hapus stopwords pada datamu

df_review["review"] = review
df_review.head()

### Melakukan Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# contoh
kalimat = 'Valentino Rossi tampak sangat menyesal setelah terjatuh pada lap terakhir MotoGP Prancis 2017'
katadasar = stemmer.stem(kalimat)
 
print(katadasar)

In [ ]:
# implementasi pada data kita
review = []
# TODO: lakukan stemming pada datamu

df_review["review"] = review
df_review.head()

In [ ]:
# sekarang coba kita cek frekuensi kemunculan untuk tiap kata

### Drop Column Dataframe Tidak dipakai

In [ ]:
df_preprocessed = df_review.copy()
df_preprocessed = df_preprocessed.drop(columns=['id_review', 'name'])
df_preprocessed.head()

### Bagi menjadi data train dan data test

In [ ]:
# install sklearn terlebih dahulu dengan pip install sklearn

In [ ]:
from sklearn.model_selection import train_test_split

# TODO: bagi dataset menjadi data training dan data testing

In [ ]:
# lihatlah perbandingannya untuk data train pada sentiment positive dan negative
print(train['label'].value_counts(normalize=True))

Karena jumlah review sentiment positive dan negative tidak seimbang pada data training, kita akan buat dataset menjadi seimbang dengan bootstraping.

In [ ]:
# bootstrap
t_1 = train[train['label']==0].sample(2000,replace=True)
t_2 = train[train['label']==1].sample(2000,replace=True)
training_bs = pd.concat([t_1, t_2])

print(train.shape)
print(training_bs.shape)
print(test.shape)

# sanity check 
df_review.shape[0] == (train.shape[0] + test.shape[0])

In [ ]:
print(training_bs['label'].value_counts(normalize=True))

In [ ]:
training_bs.head()

In [ ]:
# install terlebih dahulu dengan pip install wordcloud

In [ ]:
# wordcloud
from wordcloud import WordCloud

# Polarity == 0 negative
train_s0 = training_bs[training_bs["label"] == 0]
all_text_s0 = ' '.join(word for word in train_s0["review"])
wordcloud = WordCloud(colormap='Reds', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text_s0)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

# Polarity == 1 positive
train_s1 = training_bs[training_bs["label"] == 1]
all_text_s1 = ' '.join(word for word in train_s1["review"])
wordcloud = WordCloud(width=1000, height=1000, colormap='Blues', background_color='white', mode='RGBA').generate(all_text_s1)
plt.figure( figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

In [ ]:
# kita juga bisa cek top 20 kata untuk masing-masing sentiment
from collections import Counter

# TODO: tampilkan top 50 kata pada masing-masing sentiment

print(counter_s0)
print()
print(counter_s1)

### Feature Extraction
Feature extraction merupakan sebuah step untuk mentransformasi teks kedalam angka atau yang bisa disebut feature representation. Terdapat beragam teknik untuk merubah teks kedalam array, tapi pada dasarnya kata kata unik dalam corpus itu yang menjadi featurenya. Feature extraction yang umum digunakan dalam sklearn adalah Count Vectorizer dan TF-IDF.


Count Vectorizer ini mudahnya menghitung semua kejadian kata dalam satu dokumen sedangkan tf-idf memberikan bobot masing masing kata.

In [ ]:
# install terlebih dahulu sklearn dengan pip install sklearn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'Pusing satu kali',
    'Pusing dua kali',
    'Pusing tiga kali',
    'Pusing lagi']

vectorizer = CountVectorizer()

# contoh
X = vectorizer.fit_transform(corpus)
X.toarray()

In [ ]:
# implementasi pada dokumen kita
# TODO: lakukan fitur ekstraksi pada data training dan data testing

print(X_train.shape)
print(X_test.shape)

### Wrapping All Preprocessing

In [ ]:
def preprocess_data(data):
    # cleansing data
    data = cleansing(data)
    
    # hapus stopwords
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    data = stopword.remove(data)
    
    # stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    data = stemmer.stem(data)
    
    # count vectorizer
    data = vectorizer.transform([data])
    
    return data

# Training
Pada algoritma Machine Learning yang digunakan adalah Naive Bayes Classifier.

Naive Bayes classifier merupakan sebuh metode klasifikasi dengan probabilitas sederhana yang mengaplikasikan teorema Bayes dengan tidak ketergantungan (independen) yang tinggi.Model klasifikasi naive Bayes menghitung probabilitas posterior suatu kelas, berdasarkan pada distribusi kata-kata dalam dokumen. Klasifikasi naive Bayes dibangun oleh data pelatihan untuk memperkirakan probabilitas dari setiap kategori yang terdapat pada ciri dokumen yang diuji. Sistem akan dilatih dengan menggunakan data baru (data latih dan data uji) dan selanjutnya diberi tugas untuk menebak nilai fungsi target dari data tersebut.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# TODO: buat classifier dan lakukan training

# Evaluation
Dalam metode Evaluation menggunakan Confusion Matrix. Confusion matrix adalah matrix yang merepresentasikan hasil klasifikasi biner pada suatu dataset. Terdapat beberapa rumus umum yang dapat digunakan untuk menghitung performa klasifikasi. Hasil dari nilai accuracy, precision dan recall bisa ditampilkan dalam persentase.

\begin{equation*}
Precision | Accuracy = \frac{TP}{(TP + FP)}.
\end{equation*}

\begin{equation*}
Recall|Sensitivity|True Positive Rate = \frac{TP}{(TP + FN)}.
\end{equation*}

- **TP:** true positive
- **FP:** false positive
- **FN:** false negative


It is convenient to combine precision and recall into a single metric called *F1 score.*

\begin{equation*}
F_1 = \frac{TP}{\frac{1}{precision} + \frac{1}{recall}} = 2 \times \frac{precision \times recall}{precision + recall} = \frac{TP}{TP + \frac{FN + FP}{2}}.
\end{equation*}

In [ ]:
# lakukan prediksi pada data test
# TODO: lakukan prediksi pada data test

In [ ]:
# import library evaluation
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix

In [ ]:
# f1_score
f1_score(test['label'], predict)

In [ ]:
# precision score
precision_score(test['label'], predict)

In [ ]:
# recall score
recall_score(test['label'], predict)

In [ ]:
# confusion matrix
tn, fp, fn, tp = confusion_matrix(test['label'], predict).ravel()
tn, fp, fn, tp

Kita coba lakukan prediksi sentiment pada suatu data baru yang kita buat sendiri.

In [ ]:
review_positif = "barangnya bagus banget"
review_negatif = "barang jelek pelayanan"

Pda hasil, jika menghasilkan 1, maka reviewnya adalah positif. Sebalinyak jika menghasilkan 0, maka reviewnya adalah negatif.

In [ ]:
# TODO: lakukan predict pada data review_positif

In [ ]:
# TODO: lakukan predict pada data review_negatif